In [17]:
import cv2
import mediapipe as mp
import pandas as pd
import csv
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

def DicttoCSV(inpDict,csvFileName):
    with open(csvFileName, "w") as outfile:
        writerfile = csv.writer(outfile)
        writerfile.writerow(inpDict.keys())
        writerfile.writerows(zip(*inpDict.values()))

def CSVtoDict(fileName):
    outputDict = {}
    df = pd.read_csv(fileName)
    for v in df.columns:
        outputDict[int(v)] = None
        a=[]
        for i in df[v]:
            a.append(int(i))
        outputDict[int(v)] = a
    return outputDict
    
def ExtractPointsandSaveinCSV(IMAGE_FILES, width = 800, height = 600):
    BG_COLOR = (192, 192, 192)
    TestList = {}
    for b in IMAGE_FILES:
        TestList[b]=None
        
        with mp_holistic.Holistic(
            static_image_mode=True,
            model_complexity=2,
            enable_segmentation=True,
            refine_face_landmarks=True) as holistic:
            image = cv2.imread(b)
            down_points = (width, height)
            image = cv2.resize(image, down_points, interpolation= cv2.INTER_LINEAR)
            image_height, image_width, _ = image.shape
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if results.pose_landmarks:
                condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
                bg_image = np.zeros(image.shape, dtype=np.uint8)
                bg_image[:] = BG_COLOR
                annotated_image = image.copy()
                annotated_image = np.where(condition, annotated_image, bg_image)
                
                a = {11 : 11, 12 : 12,
                     13 : 13, 14 : 14,
                     15 : 15, 16 : 16,
                     23 : 23, 24 : 24,
                     25 : 25, 26 : 26,
                     27 : 27, 28 : 28}
                for i in (a.keys()):
                    key = a[i]
                    cordinates = []
                    X = int(np.round(results.pose_landmarks.landmark[key].x * width))
                    Y = int(np.round(results.pose_landmarks.landmark[key].y * height))
                    cordinates.append(X)
                    cordinates.append(Y)
                    a[i] = cordinates
                
                oldKeyName = b
                num = oldKeyName.rfind('.')
                newKeyName = oldKeyName[0:num]
                DicttoCSV(a, (newKeyName+".csv"))
               # print(newKeyName," ",a,"\n")
                
#                 TestList[b]=a
            
#         oldKeyName = b
#         num = oldKeyName.rfind('.')
#         newKeyName = oldKeyName[0:num]
#         TestList[newKeyName] = TestList.pop(oldKeyName)
#     print(TestList)
#DRAW POSE COUNTOURS ON LIVESTREAM

def restore_landmarks(pose_map,image,color,thickness,color_circle = (255,0,0), radious = 10, text = ' '):
    
    for indx, pose in enumerate(pose_map.keys()):
        cv2.circle(image, pose_map[pose], radious, color_circle, -thickness)
        if pose == 11:
            cv2.line(image, pose_map[pose], pose_map[13], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[23], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[12], color, thickness)
        elif pose == 12:
            cv2.line(image, pose_map[pose], pose_map[14], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[24], color, thickness)
        elif pose == 13:
            cv2.line(image, pose_map[pose], pose_map[15], color, thickness)
        elif pose == 14:
            cv2.line(image, pose_map[pose], pose_map[16], color, thickness)
        elif pose == 23:
            cv2.line(image, pose_map[pose], pose_map[25], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[24], color, thickness)
        elif pose == 24:
            cv2.line(image, pose_map[pose], pose_map[26], color, thickness)
        elif pose == 25:
            cv2.line(image, pose_map[pose], pose_map[27], color, thickness)
        elif pose == 26:
            cv2.line(image, pose_map[pose], pose_map[28], color, thickness)
            
            
        position = (10,50)
        cv2.putText(image,text,position, cv2.FONT_HERSHEY_SIMPLEX, 2,(209, 80, 0, 255),2)
def calculate_accurancy(standart_pose, user_pose):
    accurancy = {}
    standart_list = list(standart_pose.items())
    user_list = list(user_pose.items())
    for i in range(len(standart_pose)):
        point=standart_list[i][0]
        standart = standart_list[i][1]
        value = user_list[i][1]
        absolute = [abs(standart[0]-value[0]),abs(standart[1]-value[1])]
        try:
            a = absolute[0]/value[0]
            b = absolute[1]/value[1]
        except ZeroDivisionError:
            a = value[0]
            b = value[1]
        relative = [round((a*100),1),round((b*100),1)]
        accurancy[point]=relative
    return accurancy

#GET POSE LANDMARKS
def save_landmarks(results, width, height):
    a= results.pose_landmarks
    landmark_map = {11 : 11, 12 : 12,
                    13 : 13, 14 : 14,
                    15 : 15, 16 : 16,
                    23 : 23, 24 : 24,
                    25 : 25, 26 : 26,
                    27 : 27, 28 : 28}
    

    for i in (landmark_map.keys()):
        key = landmark_map[i]
        cordinates = []
        X = int(np.round(results.pose_landmarks.landmark[key].x * width))
        Y = int(np.round(results.pose_landmarks.landmark[key].y * height))
        cordinates.append(X)
        cordinates.append(Y)
        landmark_map[i] = cordinates
    return landmark_map

def loadDescription(fileName):
    with open(fileName) as f:
        lines = f.readlines()
    return lines

In [18]:
import tkinter as tk
from tkinter import ttk
import customtkinter
from tkinter import *
import numpy as np
import cv2
import os
from PIL import Image, ImageTk
import sys
import threading
import tkinter.ttk as ttk
from queue import Queue
from PIL import Image
from PIL import ImageTk
import tkinter as tk
from tkinter import ttk
import customtkinter
from tkinter import *
import numpy as np
import cv2
import os
from PIL import Image, ImageTk
customtkinter.set_appearance_mode("Dark")
customtkinter.set_default_color_theme("green")

In [19]:
WIDTH = 1100
HEIGHT = 600
APP_NAME = "CustomTkinter example_background_image.py"
global currentState
currentState = ["dddd"]

In [20]:
class App(customtkinter.CTk):
    def __init__(self, *args, **kwargs):
        self.AppVar = "dfdf"
        customtkinter.CTk.__init__(self, *args, **kwargs)
        self.title(APP_NAME)
        self.geometry(f"{WIDTH}x{HEIGHT}")
        self.protocol("WM_DELETE_WINDOW", self.on_closing)    
        self.resizable(False,False)  #Block window resize
        
        container = customtkinter.CTkFrame(master=self)
        container.pack(fill="both", expand=True)
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
    
    #////////////////////////////////////////// Page Switch 
        self.frames = {} 
        for F in (StartPage, Page1,ReactonBtnClick,Beginner):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row = 0, column = 0, sticky ="nsew")
            
        self.show_frame(StartPage)
  
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
    #////////////////////////////////////////// 
    
    
    def on_closing(self, event=0):
        self.destroy()
        
    def change_appearance_mode(self, new_appearance_mode):
        customtkinter.set_appearance_mode(new_appearance_mode)
        

    

In [21]:
class StartPage(customtkinter.CTkFrame):
    
    def __init__(self, parent, controller):
        self.indx = 0
   
        customtkinter.CTkFrame.__init__(self, parent)
       
        self.frame_left = customtkinter.CTkFrame(master=self, width=WIDTH-20, height = HEIGHT-20, corner_radius=20)
        self.frame_left.pack(fill="both", expand=True, padx=20, pady=20)
        
        
        
        self.button_1 = customtkinter.CTkButton(master=self.frame_left,text="Go to Learn",   width=60, height=60, command = lambda : [controller.show_frame(Page1)])
        self.button_1.grid(row=5, column=1, pady=10, padx=20)
        
        
          
        def edit(args):
            controller.AppVar = str(args)
            currentState[0]=str(args)
            print(currentState[0])
       
        self.beginer_btn = customtkinter.CTkButton(master=self.frame_left,text="Beginner",   width=60, height=60, command = lambda : [edit("Beginner"), controller.show_frame(Beginner)])
        self.beginer_btn.grid(row=5, column=2, pady=10, padx=20)
        
        self.intermediate_btn = customtkinter.CTkButton(master=self.frame_left,text="Intermediate",   width=60, height=60, command = lambda : [edit("Intermediate"), controller.show_frame(Beginner)])
        self.intermediate_btn.grid(row=5, column=3, pady=10, padx=20)
        
        self.advanced_btn = customtkinter.CTkButton(master=self.frame_left,text="Advanced",   width=60, height=60, command = lambda : [edit("Advanced"), controller.show_frame(Beginner)])
        self.advanced_btn.grid(row=5, column=4, pady=10, padx=20)
        
      
    
#         self.button_2 = customtkinter.CTkButton(master=self.frame_left,text = controller.AppVar, width=60, height=60, command= lambda:[edit(), test()])
#         self.button_2.grid(row=0, column=4, pady=10, padx=20)
#         def test():
#             self.button_2.configure(text=str(controller.AppVar))

In [22]:
class ReactonBtnClick(customtkinter.CTkFrame):
    
    def __init__(self, parent, controller):
        
        customtkinter.CTkFrame.__init__(self, parent)
        
        self.str1 = tk.StringVar() # declare StringVar()
        self.str1.set(controller.AppVar) 
        self.str1.trace('w', self.show_message)
        self.curtext = " "
    
        if controller.AppVar=='Beginner':
            self.curtext = "Beginner"
        elif controller.AppVar == "Advanced":
            self.curtext = "Advanced"
        else:
            self.curtext = "Intrmediate"
        self.frame_left = customtkinter.CTkFrame(master=self, width=WIDTH-20, height = HEIGHT-20, corner_radius=20)
        self.frame_left.pack(fill="both", expand=True, padx=20, pady=20)
        
        self.frame_left.grid_columnconfigure(1, weight=1)
        self.frame_left.grid_columnconfigure(2, weight=1)
        self.frame_left.grid_rowconfigure(0, weight=1)
        
        
        self.label_1 = customtkinter.CTkLabel(master=self.frame_left,
                                              text=self.curtext,
                                              text_font=("Roboto Medium", -16))  # font name and size in px
        self.label_1.grid(row=1, column=0, pady=10, padx=10)
        
        self.button_1 = customtkinter.CTkButton(master=self.frame_left,text=currentState[0],   width=60, height=60, command = lambda : controller.show_frame(StartPage))
        self.button_1.grid(row=0, column=1, pady=10, padx=20)
        def test():
            print(str(controller.AppVar)) 
        self.button_2 = customtkinter.CTkButton(master=self.frame_left,text=currentState[0],   width=60, height=60, command = test)
        self.button_2.grid(row=0, column=2, pady=10, padx=20)
    def show_message(self, *args): 
        self.str1.set(controller.AppVar) 
        self.lable_1.configure(text=str(str1))
        
#         def edit():
#             controller.AppVar = str(self.indx)
#             self.indx += 1
#             print(controller.AppVar)
       
    
#         self.button_2 = customtkinter.CTkButton(master=self.frame_left,text = controller.AppVar, width=60, height=60, command= lambda:[edit(), test()])
#         self.button_2.grid(row=0, column=2, pady=10, padx=20)
  

In [23]:
class Page1(customtkinter.CTkFrame):
    
    def __init__(self, parent, controller):

        customtkinter.CTkFrame.__init__(self, parent)
        self.TestList = {}
 ############################################################### 
        self.a1 = ["TreePose.jpg"]
        self.a2 = ["TreePose.jpg"]
        self.a3 = ["TreePose.jpg"]
        self.a = []
        if controller.AppVar=='Beginner':
            self.a = self.a1
        elif controller.AppVar == "Advanced":
            self.a = self.a2
        else:
            self.a = self.a3
            
        for i in self.a:
            num = i.rfind('.')
            Key = i[0:num]
            self.TestList[Key]=None
        for i in self.a:
            num = i.rfind('.')
            Key = i[0:num]
            self.TestList[Key]=CSVtoDict(Key+'.csv')
 #################################################################
        
        self.FRAME_WIDTH = 800
        self.FRAME_HEIGHT = 600
        self.is_running = False
        self.thread = None
        self.queue = Queue()
        self.photo = ImageTk.PhotoImage(Image.new("RGB", (self.FRAME_WIDTH, self.FRAME_HEIGHT), "white"))
        self.bind('<<MessageGenerated>>', self.on_next_frame)
        self.selectedPose = None
        ##List of Poses
        self.valList = []
        for i in self.TestList.keys():
            self.valList.append(str(i))
        self.grid_columnconfigure(1, weight=1)
        

        self.frame_top = customtkinter.CTkFrame(master=self, height = 40, corner_radius=0)
        self.frame_top.grid(row=0, column=1, sticky="nwne", padx = 10)

        self.frame_right = customtkinter.CTkFrame(master=self,height = HEIGHT-60,corner_radius=0)
        self.frame_right.grid(row=1, column=1, sticky="weswse", pady=10, padx =10)
        
        
        ##List of Poses Drop-down List
        self.btn_select = customtkinter.CTkOptionMenu(master=self.frame_top, corner_radius = 0, values = self.valList, width=200, command=self.change)
        self.btn_select.set("Chose a Pose")
        self.btn_select.place(relx=0.5, rely=0.5, anchor=CENTER)
        
        
        self.btn_mainMenu = customtkinter.CTkButton(master=self.frame_top,
                                                text="Main Menu",command = lambda : controller.show_frame(StartPage))
        self.btn_mainMenu.grid(row=3, column=0, pady=10, padx=20)
        
        
        self.btn_stop = customtkinter.CTkButton(self.frame_right, text="Stop",command=self.stop)
        self.btn_stop.grid(row=3, column=0, pady=10, padx=20)
        
        self.btn_start = customtkinter.CTkButton(self.frame_right, text="Start",command=self.start)
        self.btn_start.grid(row=3, column=1, pady=10, padx=20)
        
        self.view = customtkinter.CTkLabel(self.frame_right,image=self.photo)
        self.view.grid(row = 0, column = 0, padx = 10, pady = 10)
    

        self.label_1 = customtkinter.CTkLabel(master=self.frame_right,
                                              text="CustomTkinter",
                                              text_font=("Roboto Medium", -16))  # font name and size in px
        self.label_1.grid(row=1, column=0, pady=10, padx=10)
        
    def on_destroy(self):
        self.stop()
        self.after(20)
        if self.thread is not None:
            self.thread.join(0.2)
        self.winfo_toplevel().destroy()
        
        
    def start(self):
        self.is_running = True
        self.thread = threading.Thread(target=self.videoLoop, args=())
        self.thread.daemon = True
        self.thread.start()
        
        
        
    def stop(self):
        self.is_running = False
        
        
    def videoLoop(self, mirror=False):
        No=0
        cap = cv2.VideoCapture(No)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.FRAME_WIDTH)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.FRAME_HEIGHT)

        while self.is_running:
            ret, to_draw = cap.read()
            if mirror is True:
                to_draw = to_draw[:,::-1]
            image = cv2.cvtColor(to_draw, cv2.COLOR_BGR2RGB)
            image = cv2.flip(image,1)
            image = cv2.resize(image, (self.FRAME_WIDTH, self.FRAME_HEIGHT), interpolation= cv2.INTER_LINEAR)
            
            if self.selectedPose != None:
                Tree_map = self.TestList[self.selectedPose]
                with mp_holistic.Holistic(
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as holistic:
                    image.flags.writeable = False
                    results = holistic.process(image)
                    image.flags.writeable = True
                    if results.pose_landmarks:
                        mp_drawing.draw_landmarks(
                            image,
                            results.pose_landmarks,
                            mp_holistic.POSE_CONNECTIONS,
                            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                        restore_landmarks(Tree_map,image,(255,0,0),10)
                        
                        user_pose = save_landmarks(results,self.FRAME_WIDTH,self.FRAME_HEIGHT)
                        results = calculate_accurancy(Tree_map, user_pose)
                        results_list = list(results.items())
                        for i in range(len(results)):
                            a = results_list[i][1]
                            if a[0] > 0.5 and a[1] > 0.5:
                                print("Not Matched")
                                break
                            else:
                                restore_landmarks(Tree_map,image,(0,255,0),10, (0,255,0), 10, text = 'Matched')
                                print("Matched")  
            self.queue.put(image)
            self.event_generate('<<MessageGenerated>>')
        
        
        
    def on_next_frame(self, eventargs):
        if not self.queue.empty():
            image = self.queue.get()
            image = Image.fromarray(image)
            self.photo = ImageTk.PhotoImage(image)
            self.view.configure(image=self.photo)
     
            
          
   
    
    def change(self,*args):
        for arg in args:
            
            d = str(args)
            d = d.replace("'", "")
            d = d.replace(",", "")
            d = d.replace("(", "")
            d = d.replace(")", "")
            for i in self.TestList.keys():
                if d == i:
                    self.selectedPose=d
                    print(self.selectedPose)
                    break


In [24]:
class Beginner(customtkinter.CTkFrame):
    
    def __init__(self, parent, controller):
        customtkinter.CTkFrame.__init__(self, parent)
        self.btn_mainMenu = customtkinter.CTkButton(master=self,
                                                text="Main Menu",command = lambda : controller.show_frame(StartPage))
        self.btn_mainMenu.grid(row=3, column=0, pady=10, padx=20)
        
        self.label_1 = customtkinter.CTkLabel(master=self, width = 100, height =100, bg_color = "red",
                                              text="",
                                              text_font=("Roboto Medium", -16))  # font name and size in px
        self.label_1.grid(row=1, column=0, pady=10, padx=10)
        
        self.optionmenu_2 = customtkinter.CTkOptionMenu(master=self,values=["test.txt", "test1.txt"], command= self.change)
        self.optionmenu_2.grid(row=2, column=0, pady=10, padx=20, sticky="w")
        

        
        
    
    
    def change(self,*args):
        for arg in args:
            a = loadDescription(arg)
            self.label_1.configure(text = a)
            
            
           

In [25]:
app = App()
app.mainloop()

Beginner
Beginner
TreePose
TreePose
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched
Not Matched


In [31]:
import pandas as pd
dataframe1 = pd.read_excel('PosesList.xlsx')
a = dataframe1.columns.tolist()
for i in a:
    print(dataframe1[i].tolist())

['POSE1', 'POSE2', 'POSE3']
['POSE1', 'POSE2', 'POSE3']
['POSE1', 'POSE2', 'POSE3']


In [ ]:
df = pd.read_csv(fileName)
    for v in df.columns:
        outputDict[int(v)] = None
        a=[]
        for i in df[v]:
            a.append(int(i))
        outputDict[int(v)] = a
    return outputDict

In [12]:
TestList.keys()

selectedP = "TreePose"
targetP = " "
for i in (TestList.keys()):
    if selectedP == i:
        targetP = i
        break
print(targetP)


d = "./df'd'f'dfd'f"
d = d.replace("'", "")
d = d.replace(",", "")
d = d.replace("(", "")
d = d.replace(")", "")

TreePose


In [21]:

import tkinter as tk
my_w = tk.Tk()
from tkinter import *
my_w.geometry("400x150")  # Size of the window width x height
my_w.title("plus2net.com")  # Adding a title
 
e1_str=tk.StringVar() # declaring a StringVar()
e1 = tk.Entry(my_w,textvariable=e1_str,bg='yellow',font=28) # Entry box
e1.grid(row=0,column=0,padx=10,pady=30) 

l1 = tk.Label(my_w,  text='No of Chars here' ,font=28)  # added one Label 
l1.grid(row=0,column=1)

def my_upd(*args):
    l1.config(text=str(len(e1_str.get()))) # read & assign text to StringVar()

e1_str.trace('w',my_upd) # triggers on change of StringVar
my_w.mainloop()

In [ ]:
#Python Dictionary Convert to CSV file and load
def DicttoCSV(inpDict,csvFileName):
    with open(csvFileName, "w") as outfile:
        writerfile = csv.writer(outfile)
        writerfile.writerow(d.keys())
        writerfile.writerows(zip(*d.values()))

def CSVtoDict(fileName, storage):
    outputDict = {}
    df = pd.read_csv(fileName)
    for v in df.columns:
        outputDict[int(v)] = None
        a=[]
        for i in df[v]:
            a.append(int(i))
        outputDict[int(v)] = a
    storage.append(outputDict)

In [179]:
c = {11:[34,45], 12:[343,3434], 15:[24,34], 16:[456,111],56:[45,7878], 78:[89,33]}
d = {11:[34,45], 12:[343,3434], 15:[24,34], 16:[456,111],56:[45,7878], 78:[89,33]}
storage = []
title = "test1.csv"
DicttoCSV(d, title)
CSVtoDict(title, storage)
storage